<h1 style="text-align:center;font-size:36px">NOTEBOOK DE BAC A SABLE</h1>

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
from shapely import wkt
import altair as alt
import csv, re, os, statistics, filecmp, unidecode
from math import sqrt, pi, exp
from pathlib import PureWindowsPath
import Connexion_Transfert as ct
import Import_trafics as it
import Donnees_horaires as dh
import Visualisation as vi
import Outils
import Donnees_sources as ds
import sqlalchemy

pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 300)

In [5]:
with ct.ConnexionBdd('local_otv_boulot') as c : 
    ct.ogr2ogr_csv2pg(c.connstringOgr, r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\TamTam\RRN33\RRN33.csv',schema='public', 
                      table='dira_tamtam_2020',encodageClient='UTF-8', headers='YES')

debut import fichier C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\TamTam\RRN33\RRN33.csv avec shape2pg à 16:50:26 
 avec commande ogr2ogr -f "postgreSQL" --config PG_USE_COPY YES -lco "SCHEMA=public" PG:"host=localhost dbname=otv user=postgres password=postgres port=5432" C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\TamTam\RRN33\RRN33.csv -nln public.dira_tamtam_2020 -oo HEADERS=YES
Fait


In [63]:
df_explode=df[['id_comptag', 'id_cpt']].assign(tmp=df.id_cpt.str.split(',')).explode('tmp')

In [17]:
df_explode

,id_comptag,id_cpt,tmp
0,BdxMet-avenue de branne--0.4989;44.8426,789,789
1,BdxMet-cours du general de gaulle--0.6277;44.7555,"225,227",225
1,BdxMet-cours du general de gaulle--0.6277;44.7555,"225,227",227
2,BdxMet-cours henri brunet--0.5558;44.8698,"1514,1515",1514
2,BdxMet-cours henri brunet--0.5558;44.8698,"1514,1515",1515
...,...,...,...
1181,BdxMet-chemin du pas de la cote--0.5392;44.7822,"1759,1760",1759
1181,BdxMet-chemin du pas de la cote--0.5392;44.7822,"1759,1760",1760
1182,BdxMet-chemin du phare--0.6933;44.8521,1574,1574
1183,BdxMet-rue francois villon--0.514;44.8706,1668,1668


In [19]:
df_excel=pd.read_excel(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\BordeauxMetropole\historique\2019\base_comptages_routiers_finale.xlsx')

In [32]:
dfv85=df_explode.merge(df_excel.assign(IDENT=df_excel.IDENT.astype(str)), right_on='IDENT', left_on='tmp', how='inner')

In [38]:
dfv85V2=dfv85[['id_comptag', 'id_cpt', 'tmp', 'IDENT', 'V85_VL', 'V85_PL', 'annee']].copy()#.loc[]
dfv85V2['V85_PL']=dfv85V2.V85_PL.apply(lambda x : np.NaN if x in ('NS', 'NC') else float(x))
dfv85V2['V85_VL']=dfv85V2.V85_VL.apply(lambda x : np.NaN if x in ('NS', 'NC') else float(x))

In [64]:
dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_VL':np.mean, 'V85_PL':np.mean}).to_csv(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\travail\echanges\BM_V85_ponctuel.csv')

In [69]:
df_indic=pd.concat([dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_VL':np.mean}).reset_index().rename(
            columns={'V85_VL':'valeur'}).assign(indicateur='v85vl'),
          dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_PL':np.mean}).reset_index().rename(
            columns={'V85_PL':'valeur'}).assign(indicateur='v85pl')])

In [72]:
with ct.ConnexionBdd('local_otv_boulot') as c : 
    df_indic.loc[~df_indic.valeur.isna()].to_sql('vitesses_bm', c.sqlAlchemyConn)

In [75]:
with ct.ConnexionBdd('local_otv_boulot') as c : 
    dfv85V2.groupby(['id_cpt','id_comptag', 'annee']).agg({'V85_VL':np.mean, 'V85_PL':np.mean}).reset_index().to_sql('vitesses_bm_old_schema', c.sqlAlchemyConn)

In [205]:
fichierNord=r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\ASF\en_cours\TMJM2019_StJeanLuzSud.xls'
dfFichierNord=pd.read_excel(fichierNord, na_values='*', sheet_name=None)

dfFichierNord['EntreeS2'].drop(['DebutPeriode', 'FinPeriode', 'GARE ENT - Gare Péage *', 'GARE ENT - Libellé Gare Péage', 
                                'GARE ENT - Société *', 'DATE ENT - Annee Mois'], axis=1, inplace=True)
dfFichierNord['EntreeS2'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['EntreeS2']['pc_pl']=round(dfFichierNord['EntreeS2'].tmja_pl/dfFichierNord['EntreeS2'].tmja*100, 2)
dfFichierNord['EntreeS2']['mois']=[k for k in it.dico_mois.keys()]
dfFichierNord['SortieS1'].drop(['DebutPeriode', 'FinPeriode', 'GARE SOR - Gare Péage *', 'GARE SOR - Libellé Gare Péage', 
                                'GARE SOR - Société *', 'DATE SOR - Année Mois **'], axis=1, inplace=True)
dfFichierNord['SortieS1'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['SortieS1']['pc_pl']=round(dfFichierNord['SortieS1'].tmja_pl/dfFichierNord['SortieS1'].tmja*100, 2)
dfFichierNord['SortieS1']['mois']=[k for k in it.dico_mois.keys()]
dfFichierNord['EntreeS1'].drop(['Date', 'Libellé Gare', 'Données', 'Nb_JoursValides'], axis=1, inplace=True)
dfFichierNord['EntreeS1'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['EntreeS1']['pc_pl']=round(dfFichierNord['EntreeS1'].tmja_pl/dfFichierNord['EntreeS1'].tmja*100, 2)
dfFichierNord['EntreeS1']['mois']=[k for k in it.dico_mois.keys()]
dfFichierNord['SortieS2'].drop(['Date', 'Libellé Gare', 'Données', 'Nb_JoursValides'], axis=1, inplace=True)
dfFichierNord['SortieS2'].columns=['tmja', 'tmja_vl', 'tmja_pl', 'id_comptag']
dfFichierNord['SortieS2']['pc_pl']=round(dfFichierNord['SortieS2'].tmja_pl/dfFichierNord['SortieS2'].tmja*100, 2)
dfFichierNord['SortieS2']['mois']=[k for k in it.dico_mois.keys()]

dfBase=pd.concat([v for v in dfFichierNord.values()]).dropna()
dfBase['fichier']=os.path.basename(fichierNord)
dfBase['annee']='2019'
dfBase['src']='tableur Vinci'
dfBase['type_veh']='tv/pl'

In [199]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'comptage',dfBase.drop_duplicates(['id_comptag', 'src', 'annee', 'type_veh'])[['id_comptag', 'src', 'annee', 'type_veh']])

In [200]:
dfIdCptUniqs=cpt.recupererIdUniqComptage(dfBase.id_comptag.tolist(), '2019', 'local_otv_boulot').rename(columns={'id':'id_comptag_uniq'})

In [201]:
dfIdCptUniqs

,id_comptag_uniq,id_comptag,annee
0,53574,64-A63-Sortie2EntreeSens1-197+660,2019
1,53572,64-A63-Sortie2EntreeSens2-197+660,2019
2,53573,64-A63-Sortie2SortieSens1-197+660,2019
3,53575,64-A63-Sortie2SortieSens2-197+660,2019


In [209]:
IndicAgrege=dfBase.groupby(['id_comptag', 'fichier', 'annee']).agg({'tmja':'mean', 'pc_pl':'mean'}).reset_index()
IndicAgrege['tmja']=round(IndicAgrege.tmja)

In [203]:
pd.melt(IndicAgrege.merge(dfIdCptUniqs, on='id_comptag'), id_var=['id_comptag', 'annee', 'fichier'], value_vars=['tmja', 'pc_pl']

In [210]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',
               cpt.structureBddOld2NewForm(IndicAgrege, '2019',['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))

In [215]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',
        pd.melt(dfBase.merge(dfIdCptUniqs, on='id_comptag'), id_vars=['id_comptag_uniq', 'mois', 'fichier'], value_vars=['tmja','pc_pl' ], 
                var_name='indicateur', value_name='valeur'))

In [100]:
data=dfFichierNord.dropna(axis=1, how='all').dropna(axis=0, how='any')
data['annee']='2019'
data['fichier']=os.path.basename(fichierNord)
data=data.drop('Nb données trafic tous véh. valides', axis=1).rename(columns={'TMJ mensuel  tous véhicules':'tmja', 
                                                                        'TMJ mensuel véhicules courts':'tmjm_vl',
                                                                        'TMJ mensuel véhicules longs':'tmjm_pl'})
data['pc_pl']=round(data.tmjm_pl.astype(int)/data.tmja.astype(int)*100,2)
data['tmja']=round(data['tmja'])
dataAgrege=data.loc[data['Aut.']=='A63'].copy()
dataMensuel=data.loc[data['Aut.']!='A63'].copy()
dataMensuel['mois']=dataMensuel['Aut.'].apply(lambda x : [k for k, v in it.dico_mois.items() if any([x.split(' ')[0]==v[y].lower() for y in range(1,len(v))] )][0])

In [52]:
#insertion comptage
cpt=it.Comptage('toto')
cpt.insert_bdd('local_otv_boulot', 'comptage', 'comptage',cpt.creer_comptage(dataAgrege.id_comptag.tolist(),'2019', 'tableur Vinci', 'tv/pl'))

In [55]:
#insertion agrege
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',
               cpt.structureBddOld2NewForm(dataAgrege, '2019',['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))

In [48]:
#insertion mensuel
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',cpt.structureBddOld2NewForm(cpt.df_attr_mens, 
            cpt.annee,['id_comptag', 'annee', 'fichier', 'donnees_type'],['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept',
       'octo', 'nove', 'dece'], 'mensuel'))

In [106]:
dfIdCptUniqs=cpt.recupererIdUniqComptage(data.id_comptag.tolist(), '2019', 'local_otv_boulot').rename(columns={'id':'id_comptag_uniq'})

In [103]:
dataMensBdd=pd.melt(dataMensuel, id_vars=['mois', 'id_comptag', 'fichier'], value_vars=['tmja', 'pc_pl'],var_name='indicateur', value_name='valeur')

In [110]:
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',dataMensBdd.merge(dfIdCptUniqs, on='id_comptag').drop(['id_comptag','annee'], axis=1))

In [109]:
dataMensBdd.merge(dfIdCptUniqs, on='id_comptag').drop(['id_comptag','annee'], axis=1)

,mois,fichier,indicateur,valeur,id_comptag_uniq
0,aout,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,3255.00,53568
1,avri,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,2101.00,53568
2,dece,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1531.00,53568
3,fevr,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1777.00,53568
4,janv,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1637.00,53568
...,...,...,...,...,...
89,mai,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.89,53571
90,mars,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.58,53571
91,nove,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.21,53571
92,octo,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.61,53571


In [104]:
dataMensBdd

,mois,id_comptag,fichier,indicateur,valeur
0,aout,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,3255.00
1,avri,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,2101.00
2,dece,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1531.00
3,fevr,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1777.00
4,janv,64-A63-Sortie3EntreeSens1-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,tmja,1637.00
...,...,...,...,...,...
89,mai,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.89
90,mars,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.58
91,nove,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,5.21
92,octo,64-A63-Sortie3SortieSens2-192+0,TMJM 2019 St Jean de Luz Nord pour CEREMA juin...,pc_pl,4.61


# donnees Tipi

In [4]:
donneesMinutes=pd.read_csv(r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', delimiter=',')

In [39]:
#liste des stations TIPI
listIdCompteurTipi=donneesMinutes.PME_ID.unique()
#liste des stations OTV
with ct.ConnexionBdd('local_otv_boulot') as c : 
    listObsSupl=pd.read_sql("select distinct id_comptag, id_cpt, obs_supl from comptage.compteur where gestionnai='DIRA'", c.sqlAlchemyConn)
listStationOtvBrut=listObsSupl.obs_supl.apply(lambda x : x.split(';')[1].split('station : ')[1] if not pd.isnull(x) and 'EMC' not in x and ';' in x else x).unique()
listStationOtvBrut2=[e.split(',') for e in listStationOtvBrut if e and re.search('^M.*\.._.*$', e)]
listStationOtv=[a.strip().replace('_','') for b in listStationOtvBrut2 for a in b]

In [41]:
#station TIPI non présentes dans OTV : 
StationTipiHorsOTV=[s for s in listIdCompteurTipi if s not in listStationOtv]
StationOtvHorsTipi=[s for s in listStationOtv if s not in listIdCompteurTipi]
StationOtvTipi=[s for s in listIdCompteurTipi if s in listStationOtv]

In [46]:
len(StationTipiHorsOTV)

98

In [47]:
StationTipiHorsOTV

['M7777.A1',
 'M7777.B1',
 'M7777.C2',
 'M7777.D2',
 'MA279.Z1',
 'MA279.Z2',
 'MA280.Y1',
 'MA280.Y2',
 'MA281.X1',
 'MA281.X2',
 'MA301.W2',
 'MA309.V1',
 'MA309.V2',
 'MA310.U1',
 'MA310.U2',
 'MA311.T1',
 'MA311.T2',
 'MA312.S1',
 'MA312.S2',
 'MB133.I2',
 'MB133.J1',
 'MB133.J2',
 'MB133.K2',
 'MB133.L2',
 'MB133.P2',
 'MB133.Q2',
 'MB133.V2',
 'MB133.X2',
 'MB133.Y4',
 'MB133.p2',
 'MB133.t2',
 'MB133.u2',
 'MB133.v2',
 'MB233.B2',
 'MB233.D2',
 'MB233.J2',
 'MB233.N2',
 'MB233.Q2',
 'MB233.R2',
 'MB233.S4',
 'MB233.T2',
 'MB233.U1',
 'MB233.V2',
 'MB233.W2',
 'MB233.Y2',
 'MB233.Z2',
 'MB233.e3',
 'MB233.e7',
 'MB233.f1',
 'MB233.f6',
 'MB233.f7',
 'MB233.fC',
 'MB233.w2',
 'MB233.y2',
 'MB233.z2',
 'MB333.B5',
 'MB333.C2',
 'MB333.D2',
 'MB333.E1',
 'MB333.G3',
 'MB333.G5',
 'MB333.H1',
 'MB333.H2',
 'MB333.I2',
 'MB333.J1',
 'MB333.J2',
 'MB333.K2',
 'MB333.L2',
 'MB333.M2',
 'MB333.O2',
 'MB333.P2',
 'MB333.R4',
 'MB333.S4',
 'MB333.T1',
 'MB333.T2',
 'MB333.U2',
 'MB333.V2',

# donnees Jeff

In [5]:
#ouvrir le fichier en ne prenant que les lignes a partir de la 20ème
df=pd.read_csv(r'C:\Users\martin.schoreisz\Downloads\python\Viterbe-AM-ANP1-003.txt', sep="\t", skiprows=19)

In [25]:
#ajouter un attribut qui décrit la valeur suivante
df['L_suiv']=df.L.shift()
#un autre qui décrit celle précédente
df['L_prec']=df.L.shift(-1)

In [26]:
# ajouter un attribut de 'drapeau' qui permet de visualiser facilement les valeurs aberrantes
deltaTolerance = 1 # a regler selon tes valeurs
df['flag'] = df.apply(lambda x : True if (x['L_prec']+deltaTolerance)<x['L'] and (x['L_suiv']+deltaTolerance)<x['L'] else False, axis=1)

In [31]:
# export du tableau des valeurs aberrantes
df.loc[df['flag']].to_csv(r'C:\Users\martin.schoreisz\Downloads\python\valeurAberrantes.csv')
#export du tableaudes valeurs non aberrantes
df.loc[-df['flag']].to_csv(r'C:\Users\martin.schoreisz\Downloads\python\valeurHorsAberrantes.csv')

In [29]:
df

,Ech.,R,L,Ech/Nb_val,L_suiv,flag,L_prec
0,1,16.14,246.99,1/4670,NaN,False,246.70
1,2,16.17,246.70,2/4670,246.99,False,246.91
2,3,16.14,246.91,3/4670,246.70,False,247.26
3,4,16.13,247.26,4/4670,246.91,False,246.50
4,5,16.17,246.50,5/4670,247.26,False,246.32
...,...,...,...,...,...,...,...
4665,4666,14.84,218.73,4666/4670,218.51,False,218.81
4666,4667,14.84,218.81,4667/4670,218.73,False,218.50
4667,4668,14.85,218.50,4668/4670,218.81,False,218.19
4668,4669,14.88,218.19,4669/4670,218.50,False,218.96


# donnees CD16 comptages temporaires

In [22]:
cd16Temp=ds.FIM(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires\14-5.fim', gest='CD16')

AttributeError: 'FIM' object has no attribute 'ligne'

In [12]:
cd16Temp.liste_lign_titre

[[0,
  'VL',
  '1',
  [25,
   88,
   49,
   62,
   65,
   70,
   80,
   99,
   74,
   16,
   7,
   5,
   5,
   1,
   0,
   0,
   1,
   0,
   3,
   10,
   8,
   52,
   64,
   64,
   68,
   69,
   73,
   62,
   54,
   77,
   72,
   95,
   88,
   30,
   9,
   6,
   1,
   1,
   0,
   0,
   0,
   1,
   2,
   9,
   14,
   55,
   57,
   76,
   78,
   69,
   54,
   72,
   67,
   90,
   101,
   118,
   70,
   32,
   10,
   3,
   1,
   0,
   0,
   0,
   2,
   1,
   4,
   10,
   13,
   109,
   112,
   79,
   81,
   66,
   81,
   92,
   84,
   105,
   120,
   113,
   74,
   41,
   12,
   1,
   3,
   1,
   2,
   0,
   1,
   1,
   0,
   4,
   8,
   11,
   47,
   95,
   125,
   130,
   73,
   67,
   82,
   88,
   105,
   91,
   56,
   26,
   18,
   6,
   3,
   3,
   1,
   0,
   0,
   2,
   0,
   0,
   4,
   5,
   14,
   17,
   49,
   84,
   46,
   34,
   28,
   33,
   42,
   48,
   25,
   16,
   3,
   1,
   3,
   0,
   0,
   1,
   1,
   1,
   1,
   8,
   13,
   35,
   26,
   22,
   42,
   64,
   54,
